# Explore here

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from scipy.stats import shapiro

url = "https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv"
df = pd.read_csv(url)
df

df.describe()

df["BloodPressure"].value_counts()

df.isnull().sum()

def percentage_nulls(df):
    """
    This function returns a dictionary with the column and
    the porcentage of missing values
    """
    N_rows = df.shape[0]
    vars_ = {}
    for var in df.columns:
        vars_[var]=(df[var].isnull().sum() / N_rows)
    return vars_

percentage_nulls(df)

df=df.drop_duplicates().reset_index(drop = True) #eliminar duplicados

df.columns[np.array(df.dtypes == "object")] #the non numeric columns es poco util porque no hay categoricas

def binary(data):
  bin_reg = bin_reg = r"^[01](?:\.0)?\.?$"
  if str(data) == 'nan':
    return np.nan
  else:
    return bool(re.findall(bin_reg, str(data)))

def is_binary(df_, col):
  """
  to consider this as a pure binary var take into account that
  the others not reach a limit of range..
  #no puedehaber un porcentaje de varriable superior con más de un  digito!!!!!!
  """
  df = df_.copy()
  percent =  df[col].apply(binary).sum() / df[col].count()
  if percent > 0.5:
    return True
  else:
    return False

normal  = []
nonormal = []
binaries = []
cates = []

def tipo_var(df_):
  df = df_.copy()
  for col in df.columns:
    if df[col].dtypes.name=='int64' or df[col].dtypes.name == 'float64':
      if is_binary(df, col):
        binaries.append(col)
      else:
        if shapiro(df[col]).pvalue > 0.05:
          normal.append(col)
        else:
          nonormal.append(col)
    else:
      cates.append(col)
  return normal, nonormal, binaries,  cates

tipo_var(df) #el primero esta vacio es normales
#el segundo es no normales son varias aqui
#el tercero es binaria (target)
#el ultimo son categoricas (aqui vacio)

df['Outcome'].value_counts() #cuando hay que tomar accion y balancera datos?

# Regex para seleccionar columnas numéricas
numerical_columns = [col for col in df.columns if re.match(r'^[A-Za-z]+$', col) and col != 'Outcome'] #quito manualmente Target
df[numerical_columns].describe()


!pip install tableone


#matriz de correlacion
corr = df.corr(numeric_only=True) #importante si hay categoricos que este activo el parentesis
corr

sns.heatmap(corr, annot=True)

from tableone import TableOne

# Definir las columnas numéricas y categóricas (para numericas podria haber usado numerical_columns como ya estaba definida)
columns = df.columns.drop('Outcome').tolist()
categorical = []  # si tuviera variables categóricas van aquí

# Crear TableOne
table1 = TableOne(data=df, nonnormal=nonormal, categorical=categorical, groupby='Outcome', pval=True, htest_name=True) #divido entre con y sin diabetes

table1


#Separar el dataset en train y test
X = df.drop(columns=['Outcome'])
y = df['Outcome']

# 2. Dividir en train/test antes de escalar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 3. Escalar solo usando la media y std de X_train
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # transform SOLO (sin fit)

#algoritmo para arboles de decision

def grid_dt(X_train, y_train):
    model = DecisionTreeClassifier(random_state=666)

    class_weight = [{0:0.05, 1:0.95}, {0:0.1, 1:0.9}, {0:0.2, 1:0.8}]
    max_depth = [None, 3, 5, 10] #cuidado si uso solo none es una tupla de un solo valor, no una lista o un valor único. agrego 3, 5, 10
    min_samples_leaf = [5, 10, 20, 50, 100]
    criterion  = ["gini", "entropy"]

    grid = dict(
        class_weight=class_weight,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion
    )

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=grid,
        n_jobs=-1,
        cv=cv,
        scoring='f1',
        error_score=0,
        verbose=1
    )

    grid_result = grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_


#entreno modelo arbol de decision
best_dt = grid_dt(X_train_scaled, y_train)
y_pred_dt = best_dt.predict(X_test_scaled)

print(classification_report(y_test, y_pred_dt))


#Ahora voy a comparar con regresion logistca
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

def grid_search_lr(X_train, y_train):
    model = LogisticRegression(random_state=666, max_iter=1000)

    class_weight = [{0:0.05, 1:0.95}, {0:0.1, 1:0.9}, {0:0.2, 1:0.8}]
    solvers = ['liblinear']
    penalty = ['l1', 'l2']
    c_values = [10, 1.0, 0.1, 0.01, 0.001]

    grid = dict(
        solver=solvers,
        penalty=penalty,
        C=c_values,
        class_weight=class_weight
    )

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=grid,
        n_jobs=-1,
        cv=cv,
        scoring='f1',
        error_score=0,
        verbose=1
    )

    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    print("Mejores hiperparámetros:", best_params)
    return best_model


#entreno modelo de regresion logistca
best_lr = grid_search_lr(X_train_scaled, y_train)
y_pred_lr = best_lr.predict(X_test_scaled)

print(classification_report(y_test, y_pred_lr))


#ahora voy a comparar con un modelo de redes neuronales

def grid_search_mlp(X_train, y_train):
    model = MLPClassifier(max_iter=1000, random_state=666)

    hidden_layer_sizes = [(10,), (20,), (20, 10)]
    activation = ['relu', 'tanh']
    alpha = [0.0001, 0.001, 0.01]
    learning_rate_init = [0.001, 0.01]
    solver = ['adam']  # 'adam' suele ser más estable que 'sgd' sin mucha parametrización

    param_grid = dict(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        alpha=alpha,
        learning_rate_init=learning_rate_init,
        solver=solver
    )

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='f1',
        n_jobs=-1,
        cv=cv,
        error_score=0,
        verbose=1
    )

    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print("Mejores hiperparámetros:", grid_search.best_params_)
    return best_model


#entreno modelo redes neuronales
best_mlp = grid_search_mlp(X_train_scaled, y_train)
y_pred_mlp = best_mlp.predict(X_test_scaled)

print(classification_report(y_test, y_pred_mlp))


! pip install imbalanced-learn > null

# Undersampling si el ratio es mayor a 1:5
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1234)
X_res, y_res = rus.fit_resample(X, y)
X_res, y_res = rus.fit_resample(X, y)

def classification_report_to_df(y_true, y_pred):
    report_dict = classification_report(y_true, y_pred, output_dict=True)
    df_report = pd.DataFrame(report_dict).transpose()
    df_report = df_report.drop(index=['accuracy', 'macro avg', 'weighted avg'])  # Opcional
    return df_report

#grafico comparativo de metricas de los clasificadores
reportmlp = classification_report(y_test, y_pred_mlp, output_dict=True)
reportlr = classification_report(y_test, y_pred_lr, output_dict=True)
reportdt = classification_report(y_test, y_pred_dt, output_dict=True)

[[reportdt['0']['f1-score'],reportdt['1']['f1-score'],reportdt['accuracy']],
   [reportlr['0']['f1-score'],reportlr['1']['f1-score'],reportlr['accuracy']],
   [reportmlp['0']['f1-score'],reportmlp['1']['f1-score'],reportmlp['accuracy']]]

report_val=np.transpose([[reportdt['0']['f1-score'],reportdt['1']['f1-score'],reportdt['accuracy']],
   [reportlr['0']['f1-score'],reportlr['1']['f1-score'],reportlr['accuracy']],
   [reportmlp['0']['f1-score'],reportmlp['1']['f1-score'],reportmlp['accuracy']]])
report_val

classificador = ("MLP", "logistric reg", "decision tree")
classification_metrics = {
    '0 - f1score': report_val[0],
    '1 - f1score': report_val[1],
    'accuracy': report_val[2],
}

x = np.arange(len(classificador))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in classification_metrics.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_title('classification metrics by model')
ax.set_xticks(x + width, classificador)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0,1)

plt.show()

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

def grid_RandomForest(X_train, y_train):
  model = RandomForestClassifier(random_state=0)
  n_estimators = [280,250,290]
  criterion = ['gini', 'entropy', 'log_loss']
  min_samples_split = [0.12, 0.1, 0.2]
  max_depth = [10,12,15]
  grid = dict(n_estimators = n_estimators, criterion = criterion,
              min_samples_split = min_samples_split, max_depth = max_depth)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
  grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                            scoring='roc_auc',error_score='raise')
  grid_result = grid_search.fit(X_train, y_train)
  return  grid_result.best_estimator_

def grid_Adaboost(X_train, y_train):
    model = AdaBoostClassifier(random_state=1)
    n_estimators = [2, 15, 35, 50, 70, 100]
    learning_rate = np.linspace(0.01, 1, 10)
    grid = dict(n_estimators=n_estimators, learning_rate=learning_rate)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                               scoring='roc_auc', error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    return grid_result.best_estimator_

gridmodelrf = grid_RandomForest(X_train,y_train)
y_pred_rf = gridmodelrf.predict(X_test)
reportrf = classification_report(y_test,y_pred_rf)

gridmodelrf

print(reportrf)

gridmodelab = grid_Adaboost(X_train,y_train)
y_pred_ab = gridmodelab.predict(X_test)
reportab = classification_report(y_test,y_pred_ab)

print(reportab)